In [1]:
import sys
import os
#sys.path.insert(0, "/usr/src/app/app")
def import_path(fullpath):
    """ 
    Import a file with full path specification. Allows one to
    import from anywhere, something __import__ does not do. 
    """
    path, filename = os.path.split(fullpath)
    filename, ext = os.path.splitext(filename)
    sys.path.append(path)
    module = __import__(filename)
   # reload(module) # Might be out of date
    del sys.path[-1]
    return module

import_path("/usr/src/app/app")

<module 'app' from '/usr/src/app/app/__init__.py'>

In [2]:
from app.pkg.ml.try_on.preprocessing.aggregator import ClothProcessor
cp = ClothProcessor()

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
from app.pkg.ml.auto_clothing_set.autoset import LocalRecSys

from app.pkg.ml.buffer_converters import BytesConverter
from PIL import Image
from app.pkg.models.app.image_category import ImageCategory
import random
bc = BytesConverter()


DATA_PATH = "/usr/src/app/data"
def read_clothes(sub_folder):
    
    images_folder = os.path.join(DATA_PATH, sub_folder)
    assert os.path.exists(images_folder)
    im_paths = os.listdir(images_folder)
    abs_im_path = [os.path.join(images_folder, im_path) for im_path in im_paths]
    return abs_im_path


outerwear_clothes_paths = read_clothes("outerwear") # ["/usr/src/app/data/upper/b_bloose.png", "/usr/src/app/data/upper/b_jacket.png"]
upper_clothes_paths = read_clothes("upper") # [ "/usr/src/app/data/upper/t-shirt-blue.png","/usr/src/app/data/upper/t-shirt-miami.png", "/usr/src/app/data/upper/b_t-shirt-2.png", "/usr/src/app/data/upper/b_t-shirt-3.png", "/usr/src/app/data/upper/t-shirt-dc.png", "/usr/src/app/data/upper/t-shirt-dc2.png", "/usr/src/app/data/upper/t-shirt.png" ]
lower_clothes_paths = read_clothes("lower")  # ["/usr/src/app/data/lower/b_black_jeans.png", "/usr/src/app/data/lower/b_shorts.png", "/usr/src/app/data/lower/jeans-dc.png", "/usr/src/app/data/lower/shorts-dc.png" ]
dress_clothes_paths = read_clothes("dresses") # ["/usr/src/app/data/dresses/1.png", "/usr/src/app/data/dresses/2.png", "/usr/src/app/data/dresses/dress-dc.png"]

user_images = [f"{DATA_PATH}/human/brayan_krenston.png"]

def convert_cloth_to_list(x, category=None):
    res = []
    for i in x:
        image = Image.open(i)
        im_no_back = cp.model_background(image)
        im_bytes = bc.image_to_bytes(im_no_back)
        res.append(image)
    return res

def convert_user_to_dict(x):
    res = []
    for i in x:
        image = Image.open(i)
        im_no_back = cp.model_background(image)
        im_bytes = bc.image_to_bytes(im_no_back)
        res.append({'image':im_bytes,})
    return res

#        im_white_back = cp.model_background.replace_background_RGBA(im_no_back)


upper_clothes = convert_cloth_to_list(upper_clothes_paths, ImageCategory.UPPER_BODY)
lower_clothes = convert_cloth_to_list(lower_clothes_paths, ImageCategory.LOWER_BODY)
dress_clothes = convert_cloth_to_list(dress_clothes_paths, ImageCategory.DRESSES)
outerwear_clothes = convert_cloth_to_list(outerwear_clothes_paths, ImageCategory.UPPER_BODY)
user_images = convert_user_to_dict(user_images)


/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [4]:
from PIL import Image
import requests
from transformers import SamModel, SamProcessor

model = SamModel.from_pretrained("nielsr/slimsam-77-uniform").to("cuda:0")
processor = SamProcessor.from_pretrained("nielsr/slimsam-77-uniform")



In [5]:
img_url = "https://huggingface.co/ybelkada/segment-anything/resolve/main/assets/car.png"
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert("RGB")
input_points = [[[450, 600]]] # 2D localization of a window

def get_im_with_mask()
    inputs = processor(raw_image, input_points=input_points, return_tensors="pt").to("cuda")
    outputs = model(**inputs)

    masks = processor.image_processor.post_process_masks(outputs.pred_masks.cpu(), inputs["original_sizes"].cpu(), inputs["reshaped_input_sizes"].cpu())
    scores = outputs.iou_scores
    from PIL import Image
    import torch
    import torchvision.transforms as transforms

    # Загрузка изображения с помощью PIL
    image = raw_image

    # Преобразование изображения PIL в тензор PyTorch
    transform = transforms.ToTensor()
    tensor_image = transform(image)

    # Создание маски в формате torch.Tensor
    #mask = torch.ones_like(tensor_image)[:, :, 0]  # Создание маски для канала красного
    print(masks[0].shape)
    mask = masks[0][0]#.unsqueeze(2)  # Добавление размерности для канала

    # Применение маски к изображению
    masked_image = tensor_image 
    #+ mask * 0.5  # Умножение на 0.5 для получения полупрозрачности
    masked_image[mask] = 255
    # Преобразование обратно в изображение PIL
    masked_image = transforms.ToPILImage()(masked_image)

    # Сохранение результирующего изображения
    masked_image#.save('masked_image.jpg')
    